In [1]:
import numpy as np
import gym
import time
import math

In [13]:
env = gym.make("CartPole-v1")
action_space = env.action_space.n
print(action_space)

2


In [12]:
alpha = 0.1 #learning rate
max_iterations = 100000
gamma = 0.95

total = 0
tot_reward = 0
prev_reward = 0

observation = [30, 30, 50, 50]
array_win = np.array([0.25, 0.25, 0.01, 0.1])

epsilon = 1
epsilon_decay = 0.995

In [4]:
q_table = np.random.uniform(low=0, high=1, size=(observation + [env.action_space.n]))
q_table.shape

(30, 30, 50, 50, 2)

In [5]:
magic_array =  np.array([15,10,1,10])
def get_discrete_state(state):
    discrete_state = state/array_win + magic_array
    return tuple(discrete_state.astype(int))

In [15]:
for i in range(max_iterations):
    t0 = time.time()
    discrete_state = get_discrete_state(env.reset())
    done = False

    reward = 0

    if (i % 2000 == 0):
        print(f"Episode: {i}")
    
    while(not(done)):
        
        if np.random.random() > epsilon:
            action = np.argmax(q_table[discrete_state])
        else:
            action = np.random.randint(0, action_space)

        new_state, step_reward, done, info = env.step(action)

        reward += step_reward

        new_discrete_state = get_discrete_state(new_state)

        if (i % 2000 == 0):
            env.render()
        
        if not done:
            max_future_q = np.max(q_table[new_discrete_state])
            current_q = q_table[discrete_state + (action,)]
            
            new_q = (1 - alpha) * current_q + alpha * (reward + gamma * max_future_q)

            q_table[discrete_state + (action,)] = new_q
        
        discrete_state = new_discrete_state

    if epsilon > 0.05: 
        if reward > prev_reward and i > 10000:
            epsilon = math.pow(epsilon_decay, i - 10000)

            if i % 500 == 0:
                print("Epsilon: " + str(epsilon))
    
    t1 = time.time()
    iteration_time = t1-t0
    total += iteration_time

    tot_reward += reward
    prev_reward = reward

    if i % 1000 == 0: 
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = tot_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        tot_reward = 0



Episode: 0
Time Average: 0.00023599696159362792
Mean Reward: 0.01
Time Average: 0.002102958679199219
Mean Reward: 22.319
Episode: 2000
Time Average: 0.001750030517578125
Mean Reward: 22.368
Time Average: 0.0016110219955444336
Mean Reward: 21.882
Episode: 4000
Time Average: 0.0020630314350128173
Mean Reward: 21.989
Time Average: 0.0020600337982177733
Mean Reward: 21.962
Episode: 6000
Time Average: 0.0027570176124572755
Mean Reward: 22.066
Time Average: 0.0016519687175750733
Mean Reward: 22.757
Episode: 8000
Time Average: 0.0020181529521942137
Mean Reward: 22.099
Time Average: 0.0027918999195098875
Mean Reward: 22.183
Episode: 10000
Time Average: 0.0020263235569000244
Mean Reward: 22.053
Time Average: 0.0033259568214416505
Mean Reward: 43.83
Episode: 12000
Time Average: 0.003584658145904541
Mean Reward: 46.437
Time Average: 0.0031869943141937256
Mean Reward: 47.413
Episode: 14000
Time Average: 0.005552551984786987
Mean Reward: 46.685
Time Average: 0.003259023189544678
Mean Reward: 47.644

In [14]:
env.close()